# DFN model for sodium-ion batteries

In this notebook we use the DFN model to simulate sodium-ion batteries. The parameters are based on the article
> K. Chayambuka, G. Mulder, D.L. Danilov, P.H.L. Notten, Physics-based modeling of sodium-ion batteries part II. Model and validation, Electrochimica Acta 404 (2022) 139764. https://doi.org/10.1016/j.electacta.2021.139764.

However, the specific values (including the data for the interpolants) are taken from the COMSOL implementation presented in [this example](https://www.comsol.com/model/1d-isothermal-sodium-ion-battery-117341). As usual, we start by importing PyBaMM.

In [ ]:
import pybamm

We now need to define the model. In this case we take the `BasicDFN` model for sodium-ion batteries (note how it is called from the `pybamm.sodium_ion` submodule). Note that, at the moment, the model is identical to the one for lithium-ion batteries, but uses different parameter values.

In [ ]:
model = pybamm.sodium_ion.BasicDFN()

In order to replicate the results in the COMSOL example, we discharge at different C-rates and compare the solutions. We loop over the C-rate dictionary and solve the model for each C-rate. We append the solutions into a list so we can later plots the results.

In [ ]:
C_rates = [1 / 12, 5 / 12, 10 / 12, 1]
solutions = []

for C_rate in C_rates:
    sim = pybamm.Simulation(model, solver=pybamm.IDAKLUSolver(), C_rate=C_rate)
    sol = sim.solve([0, 4000 / C_rate])
    solutions.append(sol)

pybamm.dynamic_plot(solutions)

We can now perform a manual plot of voltage versus capacity, to compare the results with the COMSOL example.

In [ ]:
import matplotlib.pyplot as plt

for solution, C_rate in zip(solutions, C_rates):
    capacity = [i * 1000 for i in solution["Discharge capacity [A.h]"].entries]
    voltage = solution["Voltage [V]"].entries
    plt.plot(capacity, voltage, label=f"{(12 * C_rate)} A.m-2")

plt.xlabel("Discharge Capacity [mA.h]")
plt.ylabel("Voltage [V]");

In [ ]:
pybamm.print_citations()